In [1]:
# import xlrd
import os
import pandas as pd
import pickle
import openpyxl

# for plots
import pyparsing
import pydot

pd.set_option('display.max_columns', None)
os.chdir("/home/ylee/blast/result/")

## Convert Ensembl gene IDs to MGI IDs

In [2]:
similar_SigUpregul_pcg_set_M = {
    'ENSMUSG00000018433', 'ENSMUSG00000020246', 'ENSMUSG00000021730', 'ENSMUSG00000023868', 'ENSMUSG00000025812', 
    'ENSMUSG00000029580', 'ENSMUSG00000035783', 'ENSMUSG00000037351', 'ENSMUSG00000039234', 'ENSMUSG00000047497', 
    'ENSMUSG00000059430', 'ENSMUSG00000059895', 'ENSMUSG00000064115', 'ENSMUSG00000099083'
}
similar_SigUpregul_pcg_set_L = {
    'ENSMUSG00000020246', 'ENSMUSG00000023868', 'ENSMUSG00000035783', 'ENSMUSG00000037351',
    'ENSMUSG00000047497', 'ENSMUSG00000059430', 'ENSMUSG00000059895'
}

In [3]:
# Mohamed's Data
ensembl_to_mgi = {
    'ENSMUSG00000018433': 'MGI:1916229',
    'ENSMUSG00000020246': 'MGI:1915183',
    'ENSMUSG00000021730': 'MGI:1096392',
    'ENSMUSG00000023868': 'MGI:1345143',
    'ENSMUSG00000025812': 'MGI:2135608',
    'ENSMUSG00000029580': 'MGI:87904',
    'ENSMUSG00000035783': 'MGI:87909',
    'ENSMUSG00000037351': 'MGI:1917446',
    'ENSMUSG00000039234': 'MGI:1916858',
    'ENSMUSG00000047497': 'MGI:2146046',
    'ENSMUSG00000059430': 'MGI:104589',
    'ENSMUSG00000059895': 'MGI:1277098',
    'ENSMUSG00000064115': 'MGI:2442722',
    'ENSMUSG00000099083': 'MGI:2443472'
}

In [4]:
# Lihan's Data
ensembl_to_mgi_L = {
    'ENSMUSG00000020246': 'MGI:1915183',
    'ENSMUSG00000023868': 'MGI:1345143',
    'ENSMUSG00000035783': 'MGI:87909',
    'ENSMUSG00000037351': 'MGI:1917446',
    'ENSMUSG00000047497': 'MGI:2146046',
    'ENSMUSG00000059430': 'MGI:104589',
    'ENSMUSG00000059895': 'MGI:1277098'
}

In [5]:
Similar_SigUpregul_pcg_MGIset = set()
for i in similar_SigUpregul_pcg_set_L:
    Similar_SigUpregul_pcg_MGIset.add(ensembl_to_mgi_L[i])

len(Similar_SigUpregul_pcg_MGIset)

7

## Function of Genes

In [6]:
# Download Ontologies
from goatools.base import download_go_basic_obo
obo_fname = download_go_basic_obo()

# Download Associations
from goatools.base import download_ncbi_associations
fin_gene2go = download_ncbi_associations()

  EXISTS: go-basic.obo
  EXISTS: gene2go


In [7]:
from goatools.obo_parser import GODag
from goatools.associations import read_gaf

# from goatools.goea.go_enrichment_ns import GOEnrichmentStudy
from goatools.go_enrichment import GOEnrichmentStudy

# Download Ontologies
go = GODag("/home/ylee/go-basic.obo")
# Download Associations
go_associations = read_gaf("/home/ylee/mgi.gaf")

# Background gene set (all genes from the GO annotation file)
background_genes = set(go_associations.keys())
# Perform GO enrichment
goea = GOEnrichmentStudy(
    background_genes,   # Background gene set
    go_associations,    # GO annotations
    go,                 # GO ontologies
    methods=['fdr_bh']  # methods = ["bonferroni", "sidak", "holm", "fdr"] for p-value correction
)

# Run GO enrichment analysis
go_results = goea.run_study(Similar_SigUpregul_pcg_MGIset)

# Print significant results
# goea.print_results(go_results, pval=0.05)

/home/ylee/go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms
HMS:0:00:10.594694 530,650 annotations READ: /home/ylee/mgi.gaf 
19623 IDs in loaded association branch, BP

Load  Ontology Enrichment Analysis ...
Propagating term counts up: is_a
100% 19,623 of 19,623 population items found in association

Runing  Ontology Analysis: current study set of 7 IDs.
100%      5 of      5 study items found in association
 71%      5 of      7 study items found in population(19623)
Calculating 15,140 uncorrected p-values using fisher_scipy_stats
  15,140 terms are associated with 19,623 of 19,623 population items
     231 terms are associated with      5 of      7 study items
  METHOD fdr_bh:
       0 GO terms found significant (< 0.05=alpha) (  0 enriched +   0 purified): statsmodels fdr_bh
       0 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)


In [16]:
for gene in Similar_SigUpregul_pcg_MGIset:
    if gene in go_associations:
        # Get associated GO terms
        go_terms = go_associations[gene]
        
        print(f"GO terms for gene {gene}:")
        
        for go_term in go_terms:
            # Get GO term details from the ontology
            go_detail = go[go_term]
            print(f"GO Term: {go_term}, Name: {go_detail.name}, Namespace: {go_detail.namespace}")
    else:
        print(f"No GO terms found for gene {gene}")

GO terms for gene MGI:2146046:
GO Term: GO:0097435, Name: supramolecular fiber organization, Namespace: biological_process
GO Term: GO:0023051, Name: regulation of signaling, Namespace: biological_process
GO Term: GO:0030198, Name: extracellular matrix organization, Namespace: biological_process
GO Term: GO:0006029, Name: proteoglycan metabolic process, Namespace: biological_process
GO Term: GO:0010648, Name: negative regulation of cell communication, Namespace: biological_process
GO Term: GO:2001113, Name: negative regulation of cellular response to hepatocyte growth factor stimulus, Namespace: biological_process
GO Term: GO:0001503, Name: ossification, Namespace: biological_process
GO Term: GO:0009100, Name: glycoprotein metabolic process, Namespace: biological_process
GO Term: GO:0051239, Name: regulation of multicellular organismal process, Namespace: biological_process
GO Term: GO:0006508, Name: proteolysis, Namespace: biological_process
GO Term: GO:0048519, Name: negative regulat

In [19]:
gene_mgi = 'MGI:104589'  # MGI ID for Actg2

# Check if the gene MGI:104589 is in the GO associations
if gene_mgi in go_associations:
    print(f"GO terms for gene {gene_mgi}:")
    
    # Get the associated GO terms
    for go_term in go_associations[gene_mgi]:
        go_term_detail = go[go_term]
        print(f"GO Term: {go_term}, Name: {go_term_detail.name}, Namespace: {go_term_detail.namespace}")
else:
    print(f"No GO terms found for gene {gene_mgi}.")

No GO terms found for gene MGI:104589.


In [21]:
gene_mgi = 'MGI:104589'  # MGI ID for Actg2
if gene_mgi in go_associations:
    print(f"GO terms for {gene_mgi}: {go_associations[gene_mgi]}")
else:
    print(f"No GO terms found for {gene_mgi}.")

No GO terms found for MGI:104589.


In [10]:
enrichment_set = set()
for res in go_results:
    if res.p_fdr_bh < 0.05:                                # adjusted p-value threshold
        print(res.GO, res.p_fdr_bh, res.enrichment)

In [20]:
enrichment_set

{'e', 'p'}

In [18]:
go_results[1].enrichment

'e'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Convert GO enrichment results into a DataFrame
go_data = []
for rec in go_results:
    go_data.append({
        'GO_id': rec.GO,
        'GO_name': rec.name,
        'p_value': rec.get_pvalue(),  # Can choose different p-values ex) p_uncorrected, p_fdr_bh
        'enrichment': rec.enrichment  # 'e' enriched, 'p' depleted
    })

df_go = pd.DataFrame(go_data)

# Sort the data by p-value and select the top 20 GO terms
df_go = df_go[df_go.p_value < 0.05]

# .sort_values('p_value').tail(10)
# df_go = df_go.sort_values('p_value').tail(10)

# Create a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='p_value', y='GO_name', data=df_go, palette='viridis')
plt.title('Top 20 Enriched GO Terms')
plt.xlabel('P-value')
plt.ylabel('GO Terms')
plt.tight_layout()
plt.show()

## Use Ensembl API

In [8]:
import requests

# Define gene ID and API endpoint
gene_id = "ENSG00000139618"  # Example gene BRCA1
url = f"https://rest.ensembl.org/overlap/id/{gene_id}?feature=gene;content-type=application/json"

# Fetch gene information
response = requests.get(url, headers={"Content-Type": "application/json"})

# Parse and print GO terms
if response.ok:
    data = response.json()
    for feature in data:
        if "Ontology_term" in feature:
            go_terms = feature["Ontology_term"]
            for term in go_terms:
                print(term)
else:
    print(f"Error: {response.status_code}")

In [15]:
response.json()

[{'gene_id': 'ENSG00000189167',
  'biotype': 'protein_coding',
  'end': 32315363,
  'canonical_transcript': 'ENST00000533490.7',
  'description': 'zygote arrest 1 like [Source:HGNC Symbol;Acc:HGNC:37116]',
  'external_name': 'ZAR1L',
  'id': 'ENSG00000189167',
  'logic_name': 'ensembl_havana_gene_homo_sapiens',
  'start': 32303699,
  'seq_region_name': '13',
  'assembly_name': 'GRCh38',
  'strand': -1,
  'feature_type': 'gene',
  'version': 12,
  'source': 'ensembl_havana'},
 {'external_name': 'IFIT1P1',
  'description': 'interferon induced protein with tetratricopeptide repeats 1 pseudogene 1 [Source:HGNC Symbol;Acc:HGNC:5408]',
  'end': 32386108,
  'canonical_transcript': 'ENST00000400497.2',
  'biotype': 'processed_pseudogene',
  'gene_id': 'ENSG00000215515',
  'version': 2,
  'source': 'havana',
  'feature_type': 'gene',
  'strand': 1,
  'seq_region_name': '13',
  'assembly_name': 'GRCh38',
  'start': 32384660,
  'id': 'ENSG00000215515',
  'logic_name': 'havana_homo_sapiens'},
 {'e

In [ ]:
from Bio import Entrez

# Set your email (required by NCBI)
Entrez.email = "your_email@example.com"

# Search for the gene
gene_symbol = "BRCA1"
handle = Entrez.esearch(db="gene", term=gene_symbol, retmax=1)
record = Entrez.read(handle)
gene_id = record["IdList"][0]

# Fetch the gene record
handle = Entrez.efetch(db="gene", id=gene_id, retmode="xml")
gene_record = Entrez.read(handle)

# Extract GO terms from the gene record
for item in gene_record[0]["Entrezgene_properties"]["Gene-commentary_properties"]:
    if "GO term" in item["Gene-commentary_text"]:
        print(item["Gene-commentary_text"])

In [26]:
gene_record[0]["Entrezgene_properties"][1]

{'Gene-commentary_type': StringElement('254', attributes={'value': 'comment'}), 'Gene-commentary_heading': 'GeneOntology', 'Gene-commentary_source': [{'Other-source_pre-text': 'Provided by', 'Other-source_anchor': 'GOA', 'Other-source_url': 'http://www.ebi.ac.uk/GOA'}], 'Gene-commentary_comment': [{'Gene-commentary_type': StringElement('254', attributes={'value': 'comment'}), 'Gene-commentary_label': 'Function', 'Gene-commentary_comment': [{'Gene-commentary_type': StringElement('254', attributes={'value': 'comment'}), 'Gene-commentary_refs': [{'Pub_pmid': {'PubMedId': '20206173'}}], 'Gene-commentary_source': [{'Other-source_src': {'Dbtag': {'Dbtag_db': 'GO', 'Dbtag_tag': {'Object-id': {'Object-id_id': '71889'}}}}, 'Other-source_pre-text': 'enables', 'Other-source_anchor': '14-3-3 protein binding', 'Other-source_post-text': 'evidence: EXP'}]}, {'Gene-commentary_type': StringElement('254', attributes={'value': 'comment'}), 'Gene-commentary_refs': [{'Pub_pmid': {'PubMedId': '8183576'}}], 